# Movie Reviews and Bag-of-Words Modelling

🎯 The goal of this challenge is to play with the ***Bag-of-words*** modelling of texts.

✍️ In the following dataset, we have $2000$ reviews classified either as _"positive"_ or _"negative"_.

In [1]:
import pandas as pd

data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/10-Natural-Language-Processing/movie_reviews.csv")
data.head()

,target,reviews
0,neg,"plot : two teen couples go to a church party ,..."
1,neg,the happy bastard's quick movie review \ndamn ...
2,neg,it is movies like these that make a jaded movi...
3,neg,""" quest for camelot "" is warner bros . ' firs..."
4,neg,synopsis : a mentally unstable man undergoing ...


In [2]:
data.shape

(2000, 2)

In [5]:
data['target'].value_counts(normalize=True)

neg    0.5
pos    0.5
Name: target, dtype: float64

## 1. Preprocessing

❓ **Question (Cleaning Text)** ❓

- Write a function `preprocessing` that will clean a sentence and apply it to all our reviews. It should:
    - remove whitespace
    - lowercase characters
    - remove numbers
    - remove punctuation
    - tokenize
    - lemmatize
- You can store the cleaned reviews into a column called `cleaned_reviews`.
- Do not remove stopwords in this challenge, we will explain why in the section `3. N-gram modelling`

In [52]:
import string
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

def preprocessing(sentence):
    sentence= sentence.lower()
    sentence= ''.join(w for w in sentence if not w.isdigit())
    for punc in string.punctuation:
        sentence= sentence.replace(punc, ' ')
    sentence= sentence.strip()
    token= word_tokenize(sentence)
    clean_review= ''.join(WordNetLemmatizer().lemmatize(token) for token in sentence)
    return clean_review


In [53]:
# Clean reviews
data['cleaned_reviews']= data['reviews'].apply(preprocessing)

In [54]:
data.head()

,target,reviews,cleaned_reviews
0,neg,"plot : two teen couples go to a church party ,...",plot two teen couples go to a church party ...
1,neg,the happy bastard's quick movie review \ndamn ...,the happy bastard s quick movie review \ndamn ...
2,neg,it is movies like these that make a jaded movi...,it is movies like these that make a jaded movi...
3,neg,""" quest for camelot "" is warner bros . ' firs...",quest for camelot is warner bros first f...
4,neg,synopsis : a mentally unstable man undergoing ...,synopsis a mentally unstable man undergoing ...


❓ **Question (LabelEncoding)**❓

LabelEncode your target and store it into a column called `"target_encoded"`

In [55]:
from sklearn.preprocessing import LabelEncoder

In [56]:
target= data['target']
lable_encod= LabelEncoder()
data['target_encoded']= lable_encod.fit_transform(target)

In [61]:
# Quick check
data.tail()

,target,reviews,cleaned_reviews,target_encoded
1995,pos,wow ! what a movie . \nit's everything a movie...,wow what a movie \nit s everything a movie...,1
1996,pos,"richard gere can be a commanding actor , but h...",richard gere can be a commanding actor but h...,1
1997,pos,"glory--starring matthew broderick , denzel was...",glory starring matthew broderick denzel was...,1
1998,pos,steven spielberg's second epic film on world w...,steven spielberg s second epic film on world w...,1
1999,pos,"truman ( "" true-man "" ) burbank is the perfect...",truman true man burbank is the perfect...,1


## 2. Bag-of-Words Modelling

❓ **Question (NaiveBayes with unigrams)** ❓

Using `cross_validate`, score a Multinomial Naive Bayes model trained on a Bag-of-Words representation of the texts.

In [64]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_validate
from sklearn.naive_bayes import MultinomialNB

In [63]:
count_vec= CountVectorizer()
X= count_vec.fit_transform(data['cleaned_reviews'])
X.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [67]:
model= MultinomialNB()
cv_res= cross_validate(model, X.toarray(), data['target_encoded'], cv=5, n_jobs=-1, scoring='accuracy')
cv_res['test_score'].mean()

0.8130000000000001

## 3. N-gram Modelling

👀 Remember that we asked you not to remove stopwords. Why? 

👉 We will train the Naive Bayes model with bigrams. Hence, in sentence like "I do not like coriander", it is important to scan the bigram "do not" to detect negativity in this sentence for example.

❓ **Question (NaiveBayes with bigrams)** ❓

Using `cross_validate`, score a Multinomial Naive Bayes model trained on a 2-gram Bag-of-Words representation of the texts.

In [69]:
vectorizer = CountVectorizer(ngram_range = (2,2))
naivebayes = MultinomialNB()

X_bow = vectorizer.fit_transform(data.cleaned_reviews)

cv_nb = cross_validate(
    naivebayes,
    X_bow,
    data.target_encoded,
    scoring = "accuracy"
)

round(cv_nb['test_score'].mean(),2)

0.84

🏁 Congratulations! Now, you know how to train a Naive Bayes model on vectorized texts.

💾 Don't forget to `git add/commit/push` your notebook...

🚀 ... and move on to the next challenge!